In [1]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn azure-ai-ml azureml-sdk

# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed successfully!")
print("📊 Ready to start data analysis!")


✅ All packages installed successfully!
📊 Ready to start data analysis!


In [2]:
# Load your datasets with ACTUAL file names
print("📂 Loading datasets...")

# 1. Load Traffic Data (the huge 6M+ row dataset - you're using the sample)
df_traffic = pd.read_csv('../data/All Recorded Traffic.txt', sep='\t')
print(f"✅ Traffic data loaded: {df_traffic.shape[0]} rows, {df_traffic.shape[1]} columns")

# 2. Load PABT Passenger Data  
df_passenger = pd.read_csv('../data/All Recorded PABT Passenger.txt', sep='\t')
print(f"✅ Passenger data loaded: {df_passenger.shape[0]} rows, {df_passenger.shape[1]} columns")

# 3. Load PABT Bus Data
df_bus = pd.read_csv('../data/All Recorded PABT Bus.txt', sep='\t') 
print(f"✅ Bus data loaded: {df_bus.shape[0]} rows, {df_bus.shape[1]} columns")

# 4. Load Facility Mobility Speeds Data
df_speeds = pd.read_csv('../data/Facility Mobility Speeds.txt', sep='\t')
print(f"✅ Mobility speeds data loaded: {df_speeds.shape[0]} rows, {df_speeds.shape[1]} columns")

print("\n🎉 All datasets loaded successfully!")


📂 Loading datasets...
✅ Traffic data loaded: 5383378 rows, 30 columns
✅ Passenger data loaded: 2786 rows, 4 columns
✅ Bus data loaded: 2784 rows, 4 columns
✅ Mobility speeds data loaded: 192 rows, 7 columns

🎉 All datasets loaded successfully!


In [3]:
# Parse dates properly based on the data dictionary
print("📅 Parsing dates...")

# Traffic data - convert Date column
df_traffic['Date'] = pd.to_datetime(df_traffic['Date'])

# PABT Bus data - convert Start_Date and End_Date
df_bus['Start_Date'] = pd.to_datetime(df_bus['Start_Date'])
df_bus['End_Date'] = pd.to_datetime(df_bus['End_Date'])

# PABT Passenger data - convert Start_Date and End_Date  
df_passenger['Start_Date'] = pd.to_datetime(df_passenger['Start_Date'])
df_passenger['End_Date'] = pd.to_datetime(df_passenger['End_Date'])

# Facility Mobility Speeds - convert Month_Year if it exists
if 'Month_Year' in df_speeds.columns:
    df_speeds['Month_Year'] = pd.to_datetime(df_speeds['Month_Year'])

print("✅ All dates parsed successfully!")


📅 Parsing dates...


KeyError: 'Date'

In [4]:
# Check what columns actually exist in each dataset
print("🔍 CHECKING ACTUAL COLUMN NAMES")
print("="*50)

print("📊 TRAFFIC DATA COLUMNS:")
print(f"Number of columns: {len(df_traffic.columns)}")
print("Column names:", list(df_traffic.columns))
print("\nFirst few rows to understand structure:")
print(df_traffic.head(2))

print("\n" + "="*50)
print("🚌 BUS DATA COLUMNS:")
print("Column names:", list(df_bus.columns))

print("\n" + "="*50)
print("👥 PASSENGER DATA COLUMNS:")
print("Column names:", list(df_passenger.columns))

print("\n" + "="*50)
print("🚗 MOBILITY SPEEDS DATA COLUMNS:")
print("Column names:", list(df_speeds.columns))


🔍 CHECKING ACTUAL COLUMN NAMES
📊 TRAFFIC DATA COLUMNS:
Number of columns: 30
Column names: ['DAY', 'DATE', 'FAC', 'LANE', 'TIME', 'TOTAL', 'CLASS 1', 'CLASS 2', 'CLASS 3', 'CLASS 4', 'CLASS 5', 'CLASS 6', 'CLASS 7', 'CLASS 8', 'CLASS 9', 'CLASS 11', 'CASH', 'EZPASS', 'VIOLATION', 'LANEMODE', 'Month', 'FAC_B', 'Autos', 'Small_T', 'Large_T', 'Buses', 'Yr', 'FAC_G', 'FAC_G2', 'Day_Name']

First few rows to understand structure:
   DAY                     DATE  FAC  LANE  TIME  TOTAL  CLASS 1  CLASS 2  \
0    4  2022-09-15 00:00:00.000    2     6   100      1      1.0      NaN   
1    4  2022-09-15 00:00:00.000    2     6   300      1      NaN      NaN   

   CLASS 3  CLASS 4  ...  Month    FAC_B  Autos  Small_T  Large_T  Buses  \
0      NaN      NaN  ...      9  Lincoln      1        0        0      0   
1      NaN      NaN  ...      9  Lincoln      1        0        0      0   

     Yr    FAC_G   FAC_G2  Day_Name  
0  2022  Tunnels  Lincoln  Thursday  
1  2022  Tunnels  Lincoln  Thursda

In [5]:
# CORRECTED DATE PARSING with actual column names
print("📅 Parsing dates with correct column names...")

# Traffic data - use 'DATE' (uppercase)
df_traffic['DATE'] = pd.to_datetime(df_traffic['DATE'])
print("✅ Traffic dates parsed")

# PABT Bus data - convert Start_Date and End_Date
df_bus['Start_Date'] = pd.to_datetime(df_bus['Start_Date'])
df_bus['End_Date'] = pd.to_datetime(df_bus['End_Date'])
print("✅ Bus dates parsed")

# PABT Passenger data - convert Start_Date and End_Date  
df_passenger['Start_Date'] = pd.to_datetime(df_passenger['Start_Date'])
df_passenger['End_Date'] = pd.to_datetime(df_passenger['End_Date'])
print("✅ Passenger dates parsed")

# Facility Mobility Speeds - convert Month_Year
df_speeds['Month_Year'] = pd.to_datetime(df_speeds['Month_Year'])
print("✅ Mobility speeds dates parsed")

print("🎉 All dates parsed successfully!")


📅 Parsing dates with correct column names...
✅ Traffic dates parsed
✅ Bus dates parsed
✅ Passenger dates parsed
✅ Mobility speeds dates parsed
🎉 All dates parsed successfully!


In [6]:
print("=" * 60)
print("🔍 EXAMINING TRAFFIC DATA - YOUR 6M+ ROW DATASET SAMPLE")
print("=" * 60)

# Check data range and key insights
print(f"📅 Date range: {df_traffic['DATE'].min()} to {df_traffic['DATE'].max()}")
print(f"🏢 Facilities: {df_traffic['FAC_B'].unique()}")
print(f"🚨 Total violations in sample: {df_traffic['VIOLATION'].sum()}")
print(f"🚗 Total traffic volume in sample: {df_traffic['TOTAL'].sum()}")

# Key facilities analysis
print(f"\n🎯 Traffic by Facility:")
facility_summary = df_traffic.groupby('FAC_B').agg({
    'TOTAL': 'sum',
    'VIOLATION': 'sum',
    'CASH': 'sum', 
    'EZPASS': 'sum'
}).round()
print(facility_summary)

# Peak times analysis (from your sample)
print(f"\n⏰ Peak Traffic Times (sample):")
time_summary = df_traffic.groupby('TIME')['TOTAL'].sum().sort_values(ascending=False).head(10)
print(time_summary)


🔍 EXAMINING TRAFFIC DATA - YOUR 6M+ ROW DATASET SAMPLE
📅 Date range: 2013-01-01 00:00:00 to 2025-05-31 00:00:00
🏢 Facilities: ['Lincoln' 'Holland' 'Goethals' 'Bayonne' 'Outerbridge' 'GWB Upper'
 'GWB Lower' 'GWB PIP']
🚨 Total violations in sample: 81105494
🚗 Total traffic volume in sample: 1454450250

🎯 Traffic by Facility:
                 TOTAL  VIOLATION      CASH     EZPASS
FAC_B                                                 
Bayonne       38821261    1888666   4297830   32634837
GWB Lower    258169916   19417263  31556372  207196319
GWB PIP       63754929    4192434   4106239   55456274
GWB Upper    296961198   18516416  51165911  227281111
Goethals     201253919    8168861  27676826  165408487
Holland      185816893    8918942  32227218  144670811
Lincoln      228982859   13094068  26497006  189392787
Outerbridge  180689275    6908844  18229240  155551592

⏰ Peak Traffic Times (sample):
TIME
1700    85524435
1800    84489286
1600    83010550
600     81446373
1500    80484332
14

In [7]:
print("=" * 60)  
print("🔍 EXAMINING PABT PASSENGER DATA")
print("=" * 60)

print(f"📅 Date range: {df_passenger['Start_Date'].min()} to {df_passenger['Start_Date'].max()}")
print(f"🚌 Carriers: {sorted(df_passenger['Carrier'].unique())}")

# Top carriers by passenger volume[^7,^8]
print(f"\n👥 Top Carriers by Passenger Volume:")
carrier_passengers = df_passenger.groupby('Carrier')['Volume'].sum().sort_values(ascending=False)
print(carrier_passengers)

# NJ Transit dominance (as expected from data)
nj_transit_total = df_passenger[df_passenger['Carrier'] == 'NJTransit']['Volume'].sum()
total_passengers = df_passenger['Volume'].sum()
print(f"\n🎯 NJTransit Market Share: {nj_transit_total/total_passengers*100:.1f}%")


🔍 EXAMINING PABT PASSENGER DATA
📅 Date range: 2020-12-07 00:00:00 to 2025-05-26 00:00:00


TypeError: '<' not supported between instances of 'float' and 'str'

In [8]:
print("=" * 60)  
print("🔍 EXAMINING PABT PASSENGER DATA")
print("=" * 60)

print(f"📅 Date range: {df_passenger['Start_Date'].min()} to {df_passenger['Start_Date'].max()}")

# Fix the carrier sorting issue by handling NaN values
carriers_clean = df_passenger['Carrier'].dropna().unique()
print(f"🚌 Carriers: {sorted(carriers_clean)}")

# Top carriers by passenger volume - handle NaN values
print(f"\n👥 Top Carriers by Passenger Volume:")
carrier_passengers = df_passenger.groupby('Carrier', dropna=True)['Volume'].sum().sort_values(ascending=False)
print(carrier_passengers)

# NJ Transit dominance analysis - handle variations in naming
nj_transit_variations = ['NJTransit', 'NJ Transit']
nj_transit_total = 0
for variation in nj_transit_variations:
    if variation in df_passenger['Carrier'].values:
        nj_transit_total += df_passenger[df_passenger['Carrier'] == variation]['Volume'].sum()

total_passengers = df_passenger['Volume'].sum()
print(f"\n🎯 NJTransit Market Share: {nj_transit_total/total_passengers*100:.1f}%")
print(f"📊 Total Passengers (all carriers): {total_passengers:,}")


🔍 EXAMINING PABT PASSENGER DATA
📅 Date range: 2020-12-07 00:00:00 to 2025-05-26 00:00:00
🚌 Carriers: ['Academy', 'Academy ', 'C & J Bus Lines', 'C&J Bus Lines', 'Coach USA', 'DeCamp', 'Greyhound', 'HCEE - Community', 'Lakeland', 'Martz', 'NJ Transit', 'NJTransit', 'Peter Pan_Bonanza', 'Trailways', 'TransBridge']

👥 Top Carriers by Passenger Volume:
Carrier
NJTransit            12985016.0
NJ Transit            1452175.0
Coach USA              972252.0
HCEE - Community       330976.0
Greyhound              320417.4
Academy                245524.2
Lakeland               240602.0
Peter Pan_Bonanza      201468.0
Martz                  160137.0
TransBridge            129736.0
Trailways              112371.0
DeCamp                  42084.0
Academy                  5643.0
C&J Bus Lines              28.0
C & J Bus Lines             0.0
Name: Volume, dtype: float64

🎯 NJTransit Market Share: 83.9%
📊 Total Passengers (all carriers): 17,198,429.6


In [9]:
print("=" * 60)
print("🔍 EXAMINING PABT BUS DATA") 
print("=" * 60)

print(f"📅 Date range: {df_bus['Start_Date'].min()} to {df_bus['Start_Date'].max()}")

# Bus volume analysis[^7]
print(f"\n🚌 Top Carriers by Bus Count:")
carrier_buses = df_bus.groupby('Carrier')['Volume'].sum().sort_values(ascending=False)
print(carrier_buses)

# Weekly trends
print(f"\n📊 Recent vs. Early Bus Volumes:")
recent_buses = df_bus[df_bus['Start_Date'] >= '2024-01-01']['Volume'].mean()
early_buses = df_bus[df_bus['Start_Date'] < '2022-01-01']['Volume'].mean()
print(f"2024+ Average: {recent_buses:.1f} buses/week")
print(f"Pre-2022 Average: {early_buses:.1f} buses/week")
print(f"Growth: {((recent_buses/early_buses)-1)*100:.1f}%")


🔍 EXAMINING PABT BUS DATA
📅 Date range: 2020-12-07 00:00:00 to 2025-05-26 00:00:00

🚌 Top Carriers by Bus Count:
Carrier
NJ Transit           515189.0
Coach USA             32892.0
HCEE - Community      18901.0
Greyhound             12029.0
Academy                8419.0
Lakeland               7180.0
Peter Pan_Bonanza      6293.4
Trailways              4712.0
TransBridge            4413.0
Martz                  4011.0
DeCamp                 2954.0
Academy                 179.0
C & J Bus Lines          61.0
Name: Volume, dtype: float64

📊 Recent vs. Early Bus Volumes:
2024+ Average: 238.1 buses/week
Pre-2022 Average: 200.6 buses/week
Growth: 18.7%


In [10]:
print("=" * 60)
print("🔍 EXAMINING FACILITY MOBILITY SPEEDS - CONGESTION DATA")
print("=" * 60)

print(f"📅 Date range: {df_speeds['Month_Year'].min()} to {df_speeds['Month_Year'].max()}")
print(f"🏢 Facilities: {df_speeds['Facility'].unique()}")

# Congestion analysis using Delta column[^45]
print(f"\n🚨 Congestion Analysis (Delta = Avg_Speed - Freeflow):")
congestion_summary = df_speeds.groupby('Facility')['Delta'].agg(['mean', 'min', 'max']).round(1)
print(congestion_summary)

# Most congested facilities
print(f"\n🔴 Most Congested Facilities (Worst Average Delta):")
worst_congestion = df_speeds.groupby('Facility')['Delta'].mean().sort_values().head()
print(worst_congestion)

# Recent congestion trends (2025 data)[^45]
recent_congestion = df_speeds[df_speeds['Month_Year'] >= '2025-01-01']
if len(recent_congestion) > 0:
    print(f"\n📈 2025 Congestion by Facility:")
    congestion_2025 = recent_congestion.groupby('Facility')['Delta'].mean().sort_values()
    print(congestion_2025.round(1))


🔍 EXAMINING FACILITY MOBILITY SPEEDS - CONGESTION DATA
📅 Date range: 2024-01-01 00:00:00 to 2025-04-01 00:00:00
🏢 Facilities: ['BB' 'GB' 'GWB' 'HT' 'LT' 'OBX']

🚨 Congestion Analysis (Delta = Avg_Speed - Freeflow):
          mean   min   max
Facility                  
BB         2.7   0.7   4.0
GB        -0.1  -6.9   4.1
GWB      -19.0 -27.3 -11.4
HT        -8.5 -12.3  -1.3
LT       -15.3 -19.7  -9.5
OBX        1.1  -1.2   2.6

🔴 Most Congested Facilities (Worst Average Delta):
Facility
GWB   -19.009375
LT    -15.340625
HT     -8.531250
GB     -0.100000
OBX     1.125000
Name: Delta, dtype: float64

📈 2025 Congestion by Facility:
Facility
GWB   -17.4
LT    -12.6
HT     -5.6
GB      1.4
OBX     1.5
BB      2.9
Name: Delta, dtype: float64


In [11]:
# Now let's create your violation prediction analysis
print("🚨 VIOLATION RATE ANALYSIS FOR AZURE ML")
print("="*50)

# Create violation rate by facility and time for ML model
df_traffic['Hour'] = df_traffic['TIME'] // 100  # Convert to hours
df_traffic['Violation_Rate'] = (df_traffic['VIOLATION'] / df_traffic['TOTAL']) * 100

# Key features for Azure AutoML
ml_features = df_traffic.groupby(['FAC_B', 'Hour', 'Day_Name', 'Month']).agg({
    'Violation_Rate': 'mean',
    'TOTAL': 'sum',
    'VIOLATION': 'sum',
    'EZPASS': 'sum',
    'CASH': 'sum'
}).reset_index()

print(f"📊 ML Training Dataset Shape: {ml_features.shape}")
print(f"🎯 Target Variable: Violation_Rate")
print(f"🔧 Features: Facility, Hour, Day, Month, Payment Methods")

# Show sample for ML
print("\n📋 Sample Data for Azure AutoML:")
print(ml_features.head())

# Export for Azure ML
ml_features.to_csv('../data/traffic_ml_features.csv', index=False)
print("\n✅ Data exported for Azure AutoML: traffic_ml_features.csv")


🚨 VIOLATION RATE ANALYSIS FOR AZURE ML
📊 ML Training Dataset Shape: (16128, 9)
🎯 Target Variable: Violation_Rate
🔧 Features: Facility, Hour, Day, Month, Payment Methods

📋 Sample Data for Azure AutoML:
     FAC_B  Hour Day_Name  Month  Violation_Rate  TOTAL  VIOLATION  EZPASS  \
0  Bayonne     0   Friday      1        7.791155   4364        332    3409   
1  Bayonne     0   Friday      2        5.758412   3948        219    3172   
2  Bayonne     0   Friday      3        4.959349   4920        260    3881   
3  Bayonne     0   Friday      4        4.983574   4756        221    3798   
4  Bayonne     0   Friday      5        5.716101   5274        352    4061   

   CASH  
0   623  
1   557  
2   779  
3   737  
4   861  

✅ Data exported for Azure AutoML: traffic_ml_features.csv


In [4]:
# Import pandas (if not already done)
import pandas as pd

# --- Step 1: LOAD THE FULL TRAFFIC DATASET ---
# *** Using the FULL ABSOLUTE PATH you provided ***
# The r'...' syntax handles Windows backslashes correctly
print("Loading the full traffic dataset using the absolute path (this might take a moment)...")
file_path = r'C:\Users\kanha\OneDrive\Desktop\PANYNJ_Project\data\All Recorded Traffic.txt'
try:
    df_traffic = pd.read_csv(file_path, sep='\t', parse_dates=['DATE'])
    print("Dataset loaded successfully.")

    # Ensure DATE column is datetime type
    df_traffic['DATE'] = pd.to_datetime(df_traffic['DATE'])

    # --- Step 2: AGGREGATE DATA WEEKLY BY FACILITY ---
    time_frequency = 'W' # Weekly aggregation ('W-MON' starts weeks on Monday)
    print(f"Aggregating total traffic by facility and week ({time_frequency})...")

    # Group by Facility (FAC_B) and Time (weekly), then sum TOTAL traffic
    df_timeseries = df_traffic.groupby(
        ['FAC_B', pd.Grouper(key='DATE', freq=time_frequency)]
    )['TOTAL'].sum().reset_index()
    print("Aggregation complete.")

    # --- Step 3: RENAME COLUMNS FOR AZURE AUTOML ---
    df_timeseries.rename(columns={
        'DATE': 'TimeColumn',          # The time stamp column
        'TOTAL': 'TargetColumn',       # The value we want to forecast
        'FAC_B': 'SeriesIDColumn'      # Identifies each unique time series (each facility)
    }, inplace=True)
    print("Columns renamed for AutoML.")

    # --- Step 4: SORT DATA (CRITICAL FOR TIME SERIES) ---
    df_timeseries.sort_values(by=['SeriesIDColumn', 'TimeColumn'], inplace=True)
    print("Data sorted.")

    # --- Step 5: SAVE THE TIME SERIES DATASET ---
    # Save in the main project folder (where your notebook likely is)
    output_filename = 'traffic_timeseries_weekly.csv'
    df_timeseries.to_csv(output_filename, index=False)
    print("-" * 50)
    print(f"Time series dataset ready for Azure AutoML!")
    print(f"Shape: {df_timeseries.shape}")
    print(f"Saved to: {output_filename}")
    print("\nFirst 5 rows:")
    print(df_timeseries.head())
    print("\nLast 5 rows:")
    print(df_timeseries.tail())
    print("-" * 50)

except FileNotFoundError:
    print("\n--- ERROR ---")
    print(f"Could not find the file at the specified path: {file_path}")
    print("Please double-check the path and file name.")
except KeyError as e:
    print(f"\n--- ERROR ---")
    print(f"A required column is missing: {e}")
    print("Please check the column names in 'All Recorded Traffic.txt'. Expected: 'DATE', 'FAC_B', 'TOTAL'.")
except Exception as e:
    print(f"\n--- An unexpected error occurred ---")
    print(e)

Loading the full traffic dataset using the absolute path (this might take a moment)...
Dataset loaded successfully.
Aggregating total traffic by facility and week (W)...
Aggregation complete.
Columns renamed for AutoML.
Data sorted.
--------------------------------------------------
Time series dataset ready for Azure AutoML!
Shape: (5033, 3)
Saved to: traffic_timeseries_weekly.csv

First 5 rows:
  SeriesIDColumn TimeColumn  TargetColumn
0        Bayonne 2013-01-06         48776
1        Bayonne 2013-01-13         64994
2        Bayonne 2013-01-20         64052
3        Bayonne 2013-01-27         61863
4        Bayonne 2013-02-03         64458

Last 5 rows:
     SeriesIDColumn TimeColumn  TargetColumn
5028    Outerbridge 2025-05-04        293735
5029    Outerbridge 2025-05-11        295033
5030    Outerbridge 2025-05-18        297999
5031    Outerbridge 2025-05-25        291989
5032    Outerbridge 2025-06-01        251947
--------------------------------------------------


In [5]:
# Import pandas (if not already done)
import pandas as pd

# --- Step 1: LOAD THE FULL TRAFFIC DATASET ---
# *** Using relative path '../data/' assuming notebook is in a subfolder ***
print("Loading the full traffic dataset from '../data/' folder (this might take a moment)...")
try:
    # Use the '../data/' path structure
    df_traffic = pd.read_csv('../data/All Recorded Traffic.txt', sep='\t', parse_dates=['DATE'])
    print("Dataset loaded successfully.")

    # Ensure DATE column is datetime type
    df_traffic['DATE'] = pd.to_datetime(df_traffic['DATE'])

    # --- Step 2: AGGREGATE DATA WEEKLY BY FACILITY ---
    time_frequency = 'W' # Weekly aggregation ('W-MON' starts weeks on Monday)
    print(f"Aggregating total traffic by facility and week ({time_frequency})...")

    # Group by Facility (FAC_B) and Time (weekly), then sum TOTAL traffic
    df_timeseries = df_traffic.groupby(
        ['FAC_B', pd.Grouper(key='DATE', freq=time_frequency)]
    )['TOTAL'].sum().reset_index()
    print("Aggregation complete.")

    # --- Step 3: RENAME COLUMNS FOR AZURE AUTOML ---
    df_timeseries.rename(columns={
        'DATE': 'TimeColumn',          # The time stamp column
        'TOTAL': 'TargetColumn',       # The value we want to forecast
        'FAC_B': 'SeriesIDColumn'      # Identifies each unique time series (each facility)
    }, inplace=True)
    print("Columns renamed for AutoML.")

    # --- Step 4: SORT DATA (CRITICAL FOR TIME SERIES) ---
    df_timeseries.sort_values(by=['SeriesIDColumn', 'TimeColumn'], inplace=True)
    print("Data sorted.")

    # --- Step 5: SAVE THE TIME SERIES DATASET ---
    # Save in the main project folder (using '../' to go up one level)
    output_filename = '../traffic_timeseries_weekly.csv'
    df_timeseries.to_csv(output_filename, index=False)
    print("-" * 50)
    print(f"Time series dataset ready for Azure AutoML!")
    print(f"Shape: {df_timeseries.shape}")
    print(f"Saved to: {output_filename} (in your main project folder)")
    print("\nFirst 5 rows:")
    print(df_timeseries.head())
    print("\nLast 5 rows:")
    print(df_timeseries.tail())
    print("-" * 50)

except FileNotFoundError:
    print("\n--- ERROR ---")
    print("Could not find '../data/All Recorded Traffic.txt'.")
    print("This path assumes your notebook is in a subfolder (like 'notebooks/')")
    print("Check your notebook's location and the 'data' folder's position relative to it.")
except KeyError as e:
    print(f"\n--- ERROR ---")
    print(f"A required column is missing: {e}")
    print("Please check the column names in 'All Recorded Traffic.txt'. Expected: 'DATE', 'FAC_B', 'TOTAL'.")
except Exception as e:
    print(f"\n--- An unexpected error occurred ---")
    print(e)

Loading the full traffic dataset from '../data/' folder (this might take a moment)...
Dataset loaded successfully.
Aggregating total traffic by facility and week (W)...
Aggregation complete.
Columns renamed for AutoML.
Data sorted.
--------------------------------------------------
Time series dataset ready for Azure AutoML!
Shape: (5033, 3)
Saved to: ../traffic_timeseries_weekly.csv (in your main project folder)

First 5 rows:
  SeriesIDColumn TimeColumn  TargetColumn
0        Bayonne 2013-01-06         48776
1        Bayonne 2013-01-13         64994
2        Bayonne 2013-01-20         64052
3        Bayonne 2013-01-27         61863
4        Bayonne 2013-02-03         64458

Last 5 rows:
     SeriesIDColumn TimeColumn  TargetColumn
5028    Outerbridge 2025-05-04        293735
5029    Outerbridge 2025-05-11        295033
5030    Outerbridge 2025-05-18        297999
5031    Outerbridge 2025-05-25        291989
5032    Outerbridge 2025-06-01        251947
--------------------------------

In [7]:
# Import pandas (if not already done)
import pandas as pd

# --- Step 1: LOAD THE FACILITY MOBILITY SPEEDS DATASET ---
# *** Using relative path '../data/' assuming notebook is in a subfolder ***
print("Loading the Facility Mobility Speeds dataset from '../data/' folder...")
try:
    # Use the '../data/' path structure
    df_speeds = pd.read_csv('../data/Facility Mobility Speeds.txt', sep='\t', parse_dates=['Month_Year'])
    print("Dataset loaded successfully.")

    # Ensure Month_Year is datetime type
    df_speeds['Month_Year'] = pd.to_datetime(df_speeds['Month_Year'])

    # --- Step 2: ENGINEER THE TARGET VARIABLE ---
    # Define High Congestion threshold (Delta < -10 mph)
    congestion_threshold = -10
    df_speeds['IsHighCongestion'] = (df_speeds['Delta'] < congestion_threshold).astype(int) # 1 if True (High Congestion), 0 if False

    print(f"Target variable 'IsHighCongestion' created based on Delta < {congestion_threshold}.")
    print("Class distribution (0=Low Congestion, 1=High Congestion):")
    print(df_speeds['IsHighCongestion'].value_counts(normalize=True)) # Show class distribution

    # --- Step 3: SELECT FEATURES FOR CLASSIFICATION ---
    # We will predict 'IsHighCongestion' based on other factors
    # Drop columns not needed for prediction or that directly give away the answer (like Delta, Avg_Speed)
    # Also drop Facility_Order as it's just for sorting
    features_to_keep = ['Facility', 'Month_Year', 'Direction', 'Freeflow', 'IsHighCongestion']
    df_classification = df_speeds[features_to_keep].copy()

    # Optional: Extract Month and Year from Month_Year as separate features
    df_classification['Month'] = df_classification['Month_Year'].dt.month
    df_classification['Year'] = df_classification['Month_Year'].dt.year
    df_classification.drop('Month_Year', axis=1, inplace=True) # Drop original date column if month/year are used

    print("Features selected for classification.")

    # --- Step 4: SAVE THE CLASSIFICATION DATASET ---
    # Save in the main project folder (using '../' to go up one level)
    output_filename_class = '../congestion_classification_features.csv'
    df_classification.to_csv(output_filename_class, index=False)
    print("-" * 50)
    print(f"Classification dataset ready for Azure AutoML!")
    print(f"Shape: {df_classification.shape}")
    print(f"Saved to: {output_filename_class} (in your main project folder)")
    print("\nFeatures:", [col for col in df_classification.columns if col != 'IsHighCongestion'])
    print("Target:", 'IsHighCongestion')
    print("\nFirst 5 rows:")
    print(df_classification.head())
    print("-" * 50)

except FileNotFoundError:
    print("\n--- ERROR ---")
    print("Could not find '../data/Facility Mobility Speeds.txt'.")
    print("This path assumes your notebook is in a subfolder (like 'notebooks/')")
    print("and the 'data' folder is in the parent directory ('PANYNJ_Project/').")
    print("Please check your notebook's location.")
except KeyError as e:
    print(f"\n--- ERROR ---")
    print(f"A required column is missing: {e}")
    print("Check the column names in 'Facility Mobility Speeds.txt'.")
except Exception as e:
    print(f"\n--- An unexpected error occurred ---")
    print(e)

Loading the Facility Mobility Speeds dataset from '../data/' folder...
Dataset loaded successfully.
Target variable 'IsHighCongestion' created based on Delta < -10.
Class distribution (0=Low Congestion, 1=High Congestion):
IsHighCongestion
0    0.614583
1    0.385417
Name: proportion, dtype: float64
Features selected for classification.
--------------------------------------------------
Classification dataset ready for Azure AutoML!
Shape: (192, 6)
Saved to: ../congestion_classification_features.csv (in your main project folder)

Features: ['Facility', 'Direction', 'Freeflow', 'Month', 'Year']
Target: IsHighCongestion

First 5 rows:
  Facility Direction  Freeflow  IsHighCongestion  Month  Year
0       BB        EB      44.9                 0      3  2025
1       BB        WB      45.0                 0      3  2025
2       GB        EB      44.9                 0      3  2025
3       GB        WB      44.9                 0      3  2025
4      GWB        EB      46.1                 1 

In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import requests
from datetime import datetime

# --- Configuration ---
# Date range based on your traffic data
start_date_str = '2013-01-01'
end_date_str = '2025-05-31'

# Coordinates for NYC area (approximating Port Authority facilities)
latitude = 40.75 # Near Lincoln/Holland Tunnels & PABT
longitude = -74.00

# Weather variables needed (Tmax, Tmin, Precipitation, Snow)
# API variable names: temperature_2m_max, temperature_2m_min, precipitation_sum, snowfall_sum
weather_variables = 'temperature_2m_max,temperature_2m_min,precipitation_sum,snowfall_sum'

# --- API Call ---
print(f"Fetching weather data for NYC ({latitude}, {longitude}) from {start_date_str} to {end_date_str}...")

# Construct the API URL
base_url = "https://archive-api.open-meteo.com/v1/era5"
params = {
    'latitude': latitude,
    'longitude': longitude,
    'start_date': start_date_str,
    'end_date': end_date_str,
    'daily': weather_variables,
    'timezone': 'America/New_York'
}

try:
    # Make the API request
    response = requests.get(base_url, params=params)
    response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

    # Process the JSON response
    weather_data_json = response.json()

    if 'daily' in weather_data_json:
        # Convert the 'daily' data into a pandas DataFrame
        df_weather = pd.DataFrame(weather_data_json['daily'])

        # Rename columns to be more descriptive
        df_weather.rename(columns={
            'time': 'DATE', # Match the traffic data date column name
            'temperature_2m_max': 'TempMax_C',
            'temperature_2m_min': 'TempMin_C',
            'precipitation_sum': 'Precipitation_mm',
            'snowfall_sum': 'Snowfall_cm'
        }, inplace=True)

        # Convert DATE column to datetime objects (without time)
        df_weather['DATE'] = pd.to_datetime(df_weather['DATE']).dt.date
        df_weather['DATE'] = pd.to_datetime(df_weather['DATE']) # Convert back to datetime64[ns] for merging

        # --- Save the weather dataset ---
        output_filename_weather = '../nyc_weather_2013_2025.csv'
        # Save in the main project folder
        df_weather.to_csv(output_filename_weather, index=False)

        print("-" * 50)
        print("Weather data fetched and processed successfully!")
        print(f"Shape: {df_weather.shape}")
        print(f"Saved to: {output_filename_weather} (in your main project folder)")
        print("\nFirst 5 rows:")
        print(df_weather.head())
        print("\nLast 5 rows:")
        print(df_weather.tail())
        print("\nData Summary:")
        print(df_weather.describe())
        print("-" * 50)

    else:
        print("\n--- ERROR ---")
        print("Could not find 'daily' data in the API response.")
        print("Response:", weather_data_json)

except requests.exceptions.RequestException as e:
    print("\n--- ERROR ---")
    print(f"API request failed: {e}")
except Exception as e:
    print(f"\n--- An unexpected error occurred ---")
    print(e)

Fetching weather data for NYC (40.75, -74.0) from 2013-01-01 to 2025-05-31...
--------------------------------------------------
Weather data fetched and processed successfully!
Shape: (4534, 5)
Saved to: ../nyc_weather_2013_2025.csv (in your main project folder)

First 5 rows:
        DATE  TempMax_C  TempMin_C  Precipitation_mm  Snowfall_cm
0 2013-01-01        3.0       -3.9               0.0          0.0
1 2013-01-02       -1.0       -7.6               0.0          0.0
2 2013-01-03       -0.8       -9.3               0.0          0.0
3 2013-01-04        2.1       -4.5               0.0          0.0
4 2013-01-05        4.0       -1.8               0.0          0.0

Last 5 rows:
           DATE  TempMax_C  TempMin_C  Precipitation_mm  Snowfall_cm
4529 2025-05-27       25.5       11.7               0.0          0.0
4530 2025-05-28       18.8       12.3              10.6          0.0
4531 2025-05-29       21.5       12.6               2.9          0.0
4532 2025-05-30       23.3       16

In [2]:
import pandas as pd
import numpy as np
import os

print("Starting data merging and feature engineering script...")

# --- 1. Define File Paths ---
traffic_file = '../data/All Recorded Traffic.txt'
weather_file = '../data/nyc_weather_2013_2025.csv'
output_file = '../data/violation_ml_features_v2.csv'

# --- 2. Load Datasets ---
print(f"Loading raw traffic data from {traffic_file}...")
try:
    df_traffic = pd.read_csv(traffic_file, sep='\t')
except FileNotFoundError:
    print(f"FATAL ERROR: Could not find traffic file at {traffic_file}")
    exit()

print(f"Loading weather data from {weather_file}...")
try:
    df_weather = pd.read_csv(weather_file)
except FileNotFoundError:
    print(f"FATAL ERROR: Could not find weather file at {weather_file}")
    exit()

print("Data loaded successfully.")

# --- 3. Prepare Data for Merging ---

print("Preparing traffic data...")
df_traffic.columns = df_traffic.columns.str.strip()

# --- FIX #2: Normalize the 'TIME' column ---
# Convert 100, 200, 2300 -> 1, 2, 23
print("Normalizing 'TIME' column (e.g., 100 -> 1)...")
df_traffic['TIME'] = (df_traffic['TIME'] / 100).astype(int)
# --- END FIX #2 ---

# Convert DATE column to datetime objects
df_traffic['DATE'] = pd.to_datetime(df_traffic['DATE'], errors='coerce')
df_traffic = df_traffic.dropna(subset=['DATE'])

# Create the 'merge_date' column (date only, for weather merge)
df_traffic['merge_date'] = df_traffic['DATE'].dt.date

# Create the full 'timestamp' by adding the 'TIME' (hour)
df_traffic['timestamp'] = df_traffic['DATE'] + pd.to_timedelta(df_traffic['TIME'], unit='h')


# Prepare Weather Data
print("Preparing weather data...")
df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
df_weather['merge_date'] = df_weather['DATE'].dt.date
df_weather = df_weather.drop(columns=['DATE'])

# --- 4. Merge Traffic and Weather Data ---
print("Merging traffic and weather datasets...")
df_merged = pd.merge(df_traffic, df_weather, on='merge_date', how='left')
df_merged = df_merged.dropna(subset=['TempMax_C']) 

# --- 5. Feature Engineering ---
print("Engineering features...")
df_merged['Hour'] = df_merged['TIME'] # Rename for clarity

if 'VIOLATION' not in df_merged.columns:
    print("FATAL ERROR: 'VIOLATION' column not found after merge.")
    exit()

# --- 6. Aggregate Data to Create ML-Ready File ---
print("Aggregating data by facility, hour, and weather...")

grouping_cols = [
    'FAC_B',
    'merge_date', 
    'Hour',
    'Day_Name',
    'Month',
    'TempMax_C',
    'TempMin_C',
    'Precipitation_mm',
    'Snowfall_cm'
]

aggregation = {
    'VIOLATION': 'sum'
}

df_final = df_merged.groupby(grouping_cols).agg(aggregation).reset_index()

# Rename 'VIOLATION' to 'Violation_Count'
df_final = df_final.rename(columns={'VIOLATION': 'Violation_Count'})

print("Aggregation complete.")

# --- 7. Save the New Dataset ---
print(f"Saving new ML-ready file to {output_file}...")
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
df_final.to_csv(output_file, index=False)

print("\n--- SCRIPT COMPLETE ---")
print(f"Your new file is ready at: {output_file}")
print("\nFirst 5 rows of the new data:")
print(df_final.head())

Starting data merging and feature engineering script...
Loading raw traffic data from ../data/All Recorded Traffic.txt...
Loading weather data from ../data/nyc_weather_2013_2025.csv...
Data loaded successfully.
Preparing traffic data...
Normalizing 'TIME' column (e.g., 100 -> 1)...
Preparing weather data...
Merging traffic and weather datasets...
Engineering features...
Aggregating data by facility, hour, and weather...
Aggregation complete.
Saving new ML-ready file to ../data/violation_ml_features_v2.csv...

--- SCRIPT COMPLETE ---
Your new file is ready at: ../data/violation_ml_features_v2.csv

First 5 rows of the new data:
     FAC_B  merge_date  Hour Day_Name  Month  TempMax_C  TempMin_C  \
0  Bayonne  2013-01-01     0  Tuesday      1        3.0       -3.9   
1  Bayonne  2013-01-01     1  Tuesday      1        3.0       -3.9   
2  Bayonne  2013-01-01     2  Tuesday      1        3.0       -3.9   
3  Bayonne  2013-01-01     3  Tuesday      1        3.0       -3.9   
4  Bayonne  2013

In [4]:
import pandas as pd
import numpy as np
import os

print("Starting script to create new ML-ready time series CSV...")

# --- 1. Define File Paths ---
traffic_path = '../data/traffic_timeseries_weekly.csv'
weather_path = '../data/nyc_weather_2013_2025.csv'
output_path = '../data/traffic_timeseries_for_automl.csv'

try:
    # --- 2. Load Traffic Data ---
    print(f"Loading weekly traffic data from {traffic_path}...")
    df_traffic = pd.read_csv(traffic_path)
    
    # Rename original columns for clarity
    df_traffic = df_traffic.rename(columns={
        'SeriesIDColumn': 'Facility',
        'TimeColumn': 'Date',
        'TargetColumn': 'Traffic_Count'
    })
    
    # Convert Date to datetime for merging
    df_traffic['Date'] = pd.to_datetime(df_traffic['Date'])
    print("Traffic data loaded.")

    # --- 3. Load Weather Data ---
    print(f"Loading daily weather data from {weather_path}...")
    df_weather = pd.read_csv(weather_path)
    
    # Convert DATE to datetime for resampling
    df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
    print("Weather data loaded.")

    # --- 4. Aggregate Weather to Weekly ---
    # Resample to 'W-SUN' (weekly, Sunday-ending) to match the traffic data
    print("Aggregating daily weather to weekly (Sunday-ending)...")
    df_weather_weekly = df_weather.resample('W-SUN', on='DATE').agg(
        Temp_Max_Mean=('TempMax_C', 'mean'),
        Temp_Min_Mean=('TempMin_C', 'mean'),
        Precip_Sum_mm=('Precipitation_mm', 'sum'),
        Snow_Sum_cm=('Snowfall_cm', 'sum')
    ).reset_index()
    
    # Rename the new weekly 'DATE' column to 'Date' for merging
    df_weather_weekly = df_weather_weekly.rename(columns={'DATE': 'Date'})
    print("Weather data aggregated.")

    # --- 5. Merge Traffic and Weather ---
    print("Merging traffic and weekly weather data...")
    # Use a left merge to ensure all traffic records are kept
    df_merged = pd.merge(df_traffic, df_weather_weekly, on='Date', how='left')
    
    # Fill any NaNs created by the merge (e.g., missing weather data)
    df_merged = df_merged.fillna(method='ffill').fillna(method='bfill')
    print("Data merged.")

    # --- 6. Save the New Dataset ---
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    df_merged.to_csv(output_path, index=False)
    
    print(f"\n--- SCRIPT COMPLETE ---")
    print(f"New ML-ready file for AutoML is saved at: {output_path}")
    print("\nFirst 5 rows of the new data:")
    print(df_merged.head())

except FileNotFoundError as e:
    print(f"\nFATAL ERROR: File not found.")
    print(f"Details: {e}")
    print(f"Please ensure your files are located at {traffic_path} and {weather_path}")
except Exception as e:
    print(f"\nAn error occurred: {e}")

Starting script to create new ML-ready time series CSV...
Loading weekly traffic data from ../data/traffic_timeseries_weekly.csv...
Traffic data loaded.
Loading daily weather data from ../data/nyc_weather_2013_2025.csv...
Weather data loaded.
Aggregating daily weather to weekly (Sunday-ending)...
Weather data aggregated.
Merging traffic and weekly weather data...
Data merged.

--- SCRIPT COMPLETE ---
New ML-ready file for AutoML is saved at: ../data/traffic_timeseries_for_automl.csv

First 5 rows of the new data:
  Facility       Date  Traffic_Count  Temp_Max_Mean  Temp_Min_Mean  \
0  Bayonne 2013-01-06          48776       2.100000      -4.966667   
1  Bayonne 2013-01-13          64994       7.671429      -0.242857   
2  Bayonne 2013-01-20          64052       6.342857      -1.157143   
3  Bayonne 2013-01-27          61863      -2.242857      -8.700000   
4  Bayonne 2013-02-03          64458       5.228571      -2.414286   

   Precip_Sum_mm  Snow_Sum_cm  
0            0.4         0.3

C:\Users\kanha\AppData\Local\Temp\ipykernel_30468\3101142914.py:56: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged = df_merged.fillna(method='ffill').fillna(method='bfill')


In [5]:
import pandas as pd
import numpy as np
import os

print("Starting script to create new ML-ready classification CSV...")

# --- 1. Define File Paths ---
base_features_path = '../data/congestion_classification_features.csv'
speeds_path = '../data/Facility Mobility Speeds.txt'
weather_path = '../data/nyc_weather_2013_2025.csv'
output_path = '../data/congestion_classification_for_automl.csv'

try:
    # --- 2. Load Base Datasets ---
    print(f"Loading base features from {base_features_path}...")
    df_cong = pd.read_csv(base_features_path)
    
    print(f"Loading speed data from {speeds_path}...")
    # Assuming tab-separated, adjust if necessary
    df_speeds = pd.read_csv(speeds_path, sep='\t')
    
    print(f"Loading daily weather data from {weather_path}...")
    df_weather = pd.read_csv(weather_path)
    print("All data loaded.")

    # --- 3. Prepare Speed Data ---
    # Parse 'Month_Year' to get 'Month' and 'Year' for merging
    print("Preparing speed data for merge...")
    df_speeds['Month_Year_dt'] = pd.to_datetime(df_speeds['Month_Year'])
    df_speeds['Month'] = df_speeds['Month_Year_dt'].dt.month
    df_speeds['Year'] = df_speeds['Month_Year_dt'].dt.year
    # Select only the new features we want to add
    cols_to_merge = ['Facility', 'Direction', 'Month', 'Year', 'Avg_Speed', 'Delta']
    
    # --- 4. Prepare Weather Data ---
    # Filter weather for 2025 ONLY and aggregate by Month
    print("Preparing 2025 weather data...")
    df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
    df_weather_2025 = df_weather[df_weather['DATE'].dt.year == 2025].copy()
    
    # Add 'Month' column for grouping
    df_weather_2025['Month'] = df_weather_2025['DATE'].dt.month
    
    # Aggregate daily 2025 weather to monthly
    df_weather_monthly = df_weather_2025.groupby('Month').agg(
        Temp_Max_Mean_2025=('TempMax_C', 'mean'),
        Temp_Min_Mean_2025=('TempMin_C', 'mean'),
        Precip_Sum_mm_2025=('Precipitation_mm', 'sum'),
        Snow_Sum_cm_2025=('Snowfall_cm', 'sum')
    ).reset_index()
    print("Weather data aggregated.")

    # --- 5. Merge Datasets ---
    print("Merging speed features into base features...")
    # Merge the base features with the new speed features
    df_merged = pd.merge(
        df_cong, 
        df_speeds[cols_to_merge], 
        on=['Facility', 'Direction', 'Month', 'Year'], 
        how='left'
    )
    
    print("Merging monthly weather data...")
    # Merge the result with the monthly 2025 weather
    df_final = pd.merge(
        df_merged, 
        df_weather_monthly, 
        on='Month', 
        how='left'
    )
    
    # Handle any potential NaNs from merges
    df_final = df_final.fillna(method='ffill').fillna(method='bfill')
    print("All data merged.")

    # --- 6. Save the New Dataset ---
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    df_final.to_csv(output_path, index=False)
    
    print(f"\n--- SCRIPT COMPLETE ---")
    print(f"New ML-ready file for classification is saved at: {output_path}")
    print("\nFirst 5 rows of the new data:")
    print(df_final.head())
    print("\nColumns in new file:")
    print(list(df_final.columns))

except FileNotFoundError as e:
    print(f"\nFATAL ERROR: File not found.")
    print(f"Details: {e}")
    print("Please ensure your files are located in the '../data/' directory.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

Starting script to create new ML-ready classification CSV...
Loading base features from ../data/congestion_classification_features.csv...
Loading speed data from ../data/Facility Mobility Speeds.txt...
Loading daily weather data from ../data/nyc_weather_2013_2025.csv...
All data loaded.
Preparing speed data for merge...
Preparing 2025 weather data...
Weather data aggregated.
Merging speed features into base features...
Merging monthly weather data...
All data merged.

--- SCRIPT COMPLETE ---
New ML-ready file for classification is saved at: ../data/congestion_classification_for_automl.csv

First 5 rows of the new data:
  Facility Direction  Freeflow  IsHighCongestion  Month  Year  Avg_Speed  \
0       BB        EB      44.9                 0      3  2025       46.9   
1       BB        WB      45.0                 0      3  2025       48.4   
2       GB        EB      44.9                 0      3  2025       44.6   
3       GB        WB      44.9                 0      3  2025       4

C:\Users\kanha\AppData\Local\Temp\ipykernel_30468\1435753831.py:73: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final = df_final.fillna(method='ffill').fillna(method='bfill')


In [8]:
pip install pandas scikit-learn xgboost lightgbm



   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---- ----------------------------------- 8.1/72.0 MB 46.3 MB/s eta 0:00:02
   ----------- ---------------------------- 20.4/72.0 MB 52.1 MB/s eta 0:00:01
   ------------------- -------------------- 35.4/72.0 MB 58.3 MB/s eta 0:00:01
   ---------------------------- ----------- 50.6/72.0 MB 61.0 MB/s eta 0:00:01
   ------------------------------------- -- 66.6/72.0 MB 64.5 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 65.4 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 50.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 39.7 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [xgboost]
   ---------------------------------------- 0/2 [xgboost]
   ---------------------------------------- 0/2 [xgboost]
   ---------------------------------------- 0/2 [xgboost

In [9]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def load_and_prep_data(path):
    """
    Loads data and creates date features to replicate
    Azure's TimeSeriesTransformer.
    """
    print("Loading data...")
    data = pd.read_csv(path)
    
    # Re-create Azure's TimeSeriesTransformer logic
    data['merge_date'] = pd.to_datetime(data['merge_date'])
    data['year'] = data['merge_date'].dt.year
    data['month'] = data['merge_date'].dt.month
    data['day'] = data['merge_date'].dt.day
    data['day_of_year'] = data['merge_date'].dt.dayofyear
    data['week_of_year'] = data['merge_date'].dt.isocalendar().week
    
    print("Feature engineering complete.")
    return data

def build_model_pipeline():
    """
    Builds the full scikit-learn pipeline, including preprocessing
    and the final VotingRegressor with exact hyperparameters.
    """
    print("Building model pipeline...")
    
    # --- 1. Define Preprocessing Steps ---
    
    # We group features just like the Azure script
    
    # Numerical features: weather + new date features
    numeric_features = [
        'TempMin_C', 'TempMax_C', 'Precipitation_mm', 'Snowfall_cm',
        'year', 'month', 'day', 'day_of_year', 'week_of_year'
    ]
    
    # Categorical features
    categorical_features = ['FAC_B', 'Hour', 'Day_Name']

    # Create transformers
    # (Using Median Imputer as specified in the Azure blueprint)
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # (Replaces CountVectorizer with OneHotEncoder for a standard, robust solution)
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Create the master preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop' # Drop any columns we didn't specify
    )

    # --- 2. Define Base Models (Copied from your script.py) ---
    
    # These hyperparameters are copied directly from your
    # auto-generated 'script.py' file.
    
    xgb_1 = XGBRegressor(
        base_score=0.5, booster='gbtree', colsample_bytree=0.7,
        learning_rate=0.1, max_depth=6, min_child_weight=1,
        n_estimators=100, n_jobs=0, objective='reg:squarederror',
        random_state=0, reg_alpha=0, reg_lambda=0.9, 
        subsample=0.8, tree_method='auto'
    )
    
    xgb_2 = XGBRegressor(
        base_score=0.5, booster='gbtree', colsample_bytree=0.6,
        learning_rate=0.01, max_depth=7, min_child_weight=1,
        n_estimators=25, n_jobs=0, objective='reg:squarederror',
        random_state=0, reg_alpha=0.6, reg_lambda=0.9, 
        subsample=1.0, tree_method='auto'
    )
    
    lgbm_1 = LGBMRegressor(
        boosting_type='gbdt', n_estimators=100, n_jobs=-1,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        subsample=1.0, colsample_bytree=1.0
    )
    
    xgb_3 = XGBRegressor(
        base_score=0.5, booster='gbtree', colsample_bytree=0.7,
        learning_rate=0.1, max_depth=7, min_child_weight=1,
        n_estimators=50, n_jobs=0, objective='reg:squarederror',
        random_state=0, reg_alpha=0, reg_lambda=0.3,
        subsample=0.8, tree_method='auto'
    )

    # --- 3. Create the Final VotingRegressor ---
    
    voting_model = VotingRegressor(
        estimators=[
            ('model_0', xgb_1),
            ('model_1', xgb_2),
            ('model_2', lgbm_1),
            ('model_3', xgb_3)
        ],
        weights=None # Default is equal weighting
    )

    # --- 4. Create the Full Pipeline ---
    
    final_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', voting_model)
    ])
    
    return final_pipeline

# --- Main execution ---
def main():
    DATA_PATH = '../data/violation_ml_features_v2.csv'
    TARGET_COLUMN = 'Violation_Count'
    
    # 1. Load and prep data
    data = load_and_prep_data(DATA_PATH)
    
    # 2. Define X (features) and y (target)
    y = data[TARGET_COLUMN]
    X = data.drop(columns=[TARGET_COLUMN, 'merge_date']) # Drop original date
    
    # 3. Split data for training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 4. Build the model pipeline
    model = build_model_pipeline()
    
    # 5. Train the model
    print("Training the final model...")
    model.fit(X_train, y_train)
    print("Model training complete.")
    
    # 6. Evaluate the model
    score = model.score(X_test, y_test)
    print(f"\n--- Model 1 (Violation) Re-creation Complete ---")
    print(f"R-squared (R2) score on test set: {score:.4f}")
    print(f"(The original AutoML R2 was: 0.981)")
    
    # 7. Save the final model (optional, but good practice)
    with open('violation_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("Final model saved as 'violation_model.pkl'")

if __name__ == "__main__":
    main()

Loading data...
Feature engineering complete.
Building model pipeline...
Training the final model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1304
[LightGBM] [Info] Number of data points in the train set: 663027, number of used features: 48
[LightGBM] [Info] Start training from score 97.933850
Model training complete.


C:\Users\kanha\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



--- Model 1 (Violation) Re-creation Complete ---
R-squared (R2) score on test set: 0.7565
(The original AutoML R2 was: 0.981)
Final model saved as 'violation_model.pkl'
